# Creating the Model with Latex Homework as Training Data

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 16 
block_size = 32 
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('untitled.txt', 'r', encoding='utf-8') as f:
    text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l]) 


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x) 

        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

       
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
       
        for _ in range(max_new_tokens):
            
            idx_cond = idx[:, -block_size:]
            
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :] # becomes (B, C)
            
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    
    xb, yb = get_batch('train')

    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.214115 M parameters
step 0: train loss 4.7422, val loss 4.7459
step 100: train loss 2.9792, val loss 2.9643
step 200: train loss 2.6879, val loss 2.6929
step 300: train loss 2.5303, val loss 2.5629
step 400: train loss 2.3971, val loss 2.4418
step 500: train loss 2.2573, val loss 2.3086
step 600: train loss 2.1726, val loss 2.2112
step 700: train loss 2.0606, val loss 2.1055
step 800: train loss 1.9981, val loss 2.0633
step 900: train loss 1.9115, val loss 1.9973
step 1000: train loss 1.8923, val loss 1.9564
step 1100: train loss 1.8533, val loss 1.9012
step 1200: train loss 1.7826, val loss 1.8614
step 1300: train loss 1.7452, val loss 1.8470
step 1400: train loss 1.7157, val loss 1.8191
step 1500: train loss 1.7224, val loss 1.7810
step 1600: train loss 1.6612, val loss 1.7458
step 1700: train loss 1.6300, val loss 1.7366
step 1800: train loss 1.6170, val loss 1.7261
step 1900: train loss 1.6059, val loss 1.7179
step 2000: train loss 1.5839, val loss 1.6959
step 2100: train loss 1.

## The Results

	ivent in many represtrions on well. coment wprof in $\frac{1}{n}}\int_j \mathbb{R}$ and $n \in \mathcaf{PP}L_nfu_n}(x) + U_n|f = ||A^n(\omega)|_\Ber{t\beta 0}$.
\end{nemm.


\textbf

\\bigskeul{Trandmaumum this in $F(T^k_n)-Rx\leq 1$ sunce we sapp'es}{A\phi/4}^n \quad \geq\ehreal{Sclamss wapping_A^*(\omega)\aplrown B = \{1,A^n$ and $U$, there evencensposition $(1)^n^k_j = \lamng,
\end{gather*}
\begin{equations}




\noindent
\begin{gather}
\textbf{Solutions}
\begin{mdframed}
    \textbf{Coronspace}

\Circled{retheorem}

iknary that $f(\alpha_\sigma_n)x'(x-x') = f(x,z)) \intart[Sthinumentegraphiest of maxtilly can holds for arrrowcuran coint function Weierstras}
\begin{proof}
     1.e. $A_\in \sin^1$ and $B(s) = \frac{F}{\infty}\sin\{e\pi \ZZ)\\
    A\neq 0$ such that \(n g_0 \) for all \( x = \sum_{ence U$, \( f$\omega)$ whichout quassisfuly, if \( \epsilon \), $\frac{n(x,L)$ and
it the lit e^{-ax} \qquad &\phi\frac{\pi i x}{a^2})$ and (rev+s) = 0 + x_1
(s_2')] + 1
\end{align*}
Feq b)

\nequall{A}{s\to \rho')}[F(n-k^2)$ and $\mu(C_n)$ is a \textbting \( v\\)
    ||p\left| \sup_{x_1 \in \frac{1}{n}} \quad \epsilor f_0 \implies [u_1$. we ressteps,.\
   \therefore \to B - \alpha d \ndot \text{$B_n = D$f(y), ach and 
        \]
     P_{R} U_k\\
E_{X_(\in \mathbb U_Tr(A)\omega)$, theorem}

       Now the eadite reventrol}



%\Uition{motr}
\textbf{P.otcer}
\textisfor fall graphich $\in U_x_n d_2
     \]
        = 0
\end{gather*}
The given theorem
\begin{gather*}
    We^{\pinfty}}F(s) = \frac{(e^{\omega)}{dx}
\end{gather*}
\begin{gather*}
    c_{n+st)}{y^2\beta x} = \frac{1}{n}f(a)+g(x_1) \in p_{x/2ke^2} du
\end{gather*}

\begin{gather*}
    \thereforeulation quoithmonangle

\begin{theorem}
     \item \( \frac{du}{d^*+ \frac{R}{k^2}B = \frac{1}{L}\lambda_Ex\frac{L}{2}}\lambda_Lu = \Lambda_-\frac{d}{dx}} = \frac{C(s)\cos\{\PPright] = \frac{d }{dx e^{-\sqrtack}X\right]
\]

\thereforem{prose}
\begin{gather*}
    \thereefore \qquad \omega, c) \Dela_{\varc{u_n}{n}}\superintion

#### latex has very complicated syntax so I wasn't expecting perfect results. Very happy that the model managed to create some text boxes correctly.